In [ ]:
# default_exp architecture.MSResNet

In [ ]:
#export
import torch
import torch.nn as nn
from defocus.architecture.common import *

class Generator(nn.Module):
    '''
    MSResNet Generator
    '''
    def __init__(self, n_scales = 3, n_feats=64, kernel_size=5, n_resblocks=19):
        super(Generator, self).__init__()
        
        # Seungjun does not scale the images down to [-1,1] but rather uses [-127.5, 127.5]
        # this self.mean variable will be used for that. wonder why he calls it the "mean"?
        self.mean = 255.0 / 2
        self.n_scales = n_scales 
        
        coarsest_level = ResNet(in_channels=3, 
                                out_channels=3,
                                n_feats=n_feats,
                                kernel_size=kernel_size,
                                n_resblocks=n_resblocks)
        finer_levels = [ResNet(in_channels=6,
                               out_channels=3, 
                               n_feats=n_feats, 
                               kernel_size=kernel_size, 
                               n_resblocks=n_resblocks) for _ in range(n_scales-1)]
               
        self.scale_networks = nn.ModuleList([]) 
        self.scale_networks.append(coarsest_level)
        self.scale_networks.extend(finer_levels)    
        
        # note that the original implementation always uses 5x5 kernels (default) for upsampling
        self.upconv_blocks = nn.ModuleList([UpConv2D() for _ in range(n_scales-1)])
        
    def forward(self, input_pyramid):
        
        output_pyramid = [None]*self.n_scales
        for scale in range(self.n_scales): 
            if scale == 0:
                input_ = input_pyramid[scale] - self.mean
            else:
                upconvolved_from_previous = self.upconv_blocks[scale-1](output_pyramid[scale-1])
                input_ = torch.cat((input_pyramid[scale] - self.mean, upconvolved_from_previous) ,1)
                
            output_pyramid[scale] = self.scale_networks[scale](input_)
        
        return output_pyramid       

In [ ]:
#export
import torch.nn as nn

class Discriminator(nn.Module):
    '''
    MSResNet Discriminator
    '''
    def __init__(self, n_feats=64, kernel_size=5):
        super(Discriminator, self).__init__()

        # self.args = args
#         n_feats = args.n_feats
#         kernel_size = args.kernel_size

        def conv(kernel_size, in_channel, n_feats, stride, pad=None):
            if pad is None:
                pad = (kernel_size-1)//2

            return nn.Conv2d(in_channel, n_feats, kernel_size, stride=stride, padding=pad, bias=False)

        self.conv_layers = nn.ModuleList([
            conv(kernel_size, 3,         n_feats//2, 1),    # 256
            conv(kernel_size, n_feats//2, n_feats//2, 2),   # 128
            conv(kernel_size, n_feats//2, n_feats,   1),
            conv(kernel_size, n_feats,   n_feats,   2),     # 64
            conv(kernel_size, n_feats,   n_feats*2, 1),
            conv(kernel_size, n_feats*2, n_feats*2, 4),     # 16
            conv(kernel_size, n_feats*2, n_feats*4, 1),
            conv(kernel_size, n_feats*4, n_feats*4, 4),     # 4
            conv(kernel_size, n_feats*4, n_feats*8, 1),
            conv(4,           n_feats*8, n_feats*8, 4, 0),  # 1
        ])

        self.act = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.dense = nn.Conv2d(n_feats*8, 1, 1, bias=False)

    def forward(self, x):

        for layer in self.conv_layers:
            x = self.act(layer(x))

        x = self.dense(x)

        return x

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_model.ipynb.
Converted 02_architecture_common.ipynb.
Converted 03_architecture_MSResNet.ipynb.
Converted 04_dataset_common.ipynb.
Converted 05_dataset_MSResNet.ipynb.
Converted 06_trainer.ipynb.
Converted 99_diffaugment.ipynb.
Converted trials.ipynb.
